In [1]:
import polars as pl
import xgboost as xgb
import numpy as np
import requests
import pandas as pd
import itertools
import os
import json
import sqlite3

# --- CONFIGURACIÓN ---
# Ajusta el nombre del modelo si usas la v12 o v3
MODEL_FILE = "draft_oracle_brain_v12_final.json"
FEATURE_FILE = "draft_oracle_feature_store.parquet"
PRO_SIG_FILE = "draft_oracle_pro_signatures.parquet" # <--- NUEVO
TOURNAMENT_META_FILE = "draft_oracle_tournament_meta.parquet" # <--- NUEVO
SYNERGY_FILE = "draft_oracle_synergy_matrix.parquet" # <--- NECESARIO PARA EXPLICAR COMPOSICIÓN
DB_FILE = "esports_data.db"

# Cargar las columnas que usó el modelo
with open("model_features_v12.json", "r") as f:
    features = json.load(f)

print(f"🔍 Revisando {len(features)} variables usadas...")

# Lista de palabras prohibidas que chivan el resultado
banned_words = ["win", "target", "gold_earned", "kills", "deaths", "victory", "duration"]

found_leaks = []
for col in features:
    for ban in banned_words:
        # Ojo: 'stat_winrate' es el promedio histórico (LEGAL). 'win' a secas es el resultado (ILEGAL).
        if ban == col:
            found_leaks.append(col)

if found_leaks:
    print(f"🚨 ALERTA ROJA: Se encontraron variables prohibidas: {found_leaks}")
    print("Debes borrarlas de 'feature_cols' y re-entrenar.")
else:
    print("✅ LIMPIO: No hay variables obvias de resultado directo.")
    print("El AUC de 0.78 viene de la potencia de la Matriz de Sinergias (y un poco de target encoding).")

class TournamentDraft:
    def __init__(self):
        print("⚙️  Cargando Tournament Suite V9.5 (SQL Preserved + Meta)...")

        # 1. CARGA DE MODELO
        self.model = xgb.Booster()
        try:
            self.model.load_model(MODEL_FILE)
            print(f"   🧠 Cerebro cargado: {MODEL_FILE}")
        except:
            print(f"   ❌ Error cargando {MODEL_FILE}. Verifica la ruta.")

        # 2. CARGA DE FEATURES
        self.df = pl.read_parquet(FEATURE_FILE)

        # 3. CARGA DE PROS (FACTOR HUMANO)
        if os.path.exists(PRO_SIG_FILE):
            print(f"   🏆 Base de Datos de Pros cargada: {PRO_SIG_FILE}")
            self.pro_stats = pl.read_parquet(PRO_SIG_FILE)
        else:
            print("   ⚠️ No se encontró 'draft_oracle_pro_signatures.parquet'. Modo SoloQ.")
            self.pro_stats = None

        # 4. CARGA DE META TORNEO (NUEVO BLOQUE - NO TOCA LO DEMÁS)
        if os.path.exists(TOURNAMENT_META_FILE):
            print(f"   🔥 Meta de Torneos cargado: {TOURNAMENT_META_FILE}")
            self.meta_stats = pl.read_parquet(TOURNAMENT_META_FILE)
        else:
            print("   ⚠️ No se encontró Meta de Torneos. Se ignorará este factor.")
            self.meta_stats = None

        if os.path.exists(SYNERGY_FILE):
            print("   ❤️  Matriz de Sinergias: Cargada")
            self.synergy_raw = pl.read_parquet(SYNERGY_FILE)
            # Creamos un mapa rápido para búsquedas {(id1, id2): winrate}
            self.synergy_map = {}
            rows = self.synergy_raw.select(["champ_id", "champ_id_right", "syn_winrate"]).to_numpy()
            for r in rows:
                self.synergy_map[(r[0], r[1])] = r[2]
                self.synergy_map[(r[1], r[0])] = r[2] # Bidireccional
        else:
            self.synergy_map = {}

        self._load_api()
        self._prepare_role_solver()
        self._prepare_model_cols()

        # ESTADO DEL TORNEO (MANTENIDO)
        self.series_config = {
            "mode": "NORMAL",
            "total_games": 1,
            "current_game": 1
        }
        self.history = []

        self.teams = {
            "BLUE": {"name": "Blue Team", "players": {}},
            "RED": {"name": "Red Team", "players": {}}
        }

        self.reset_game_board()


    # --- FUNCIONES DE SOPORTE (MANTENIDAS) ---
    def _load_api(self):
        print("🌍 Conectando a Riot API...")
        try:
            v = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
            r = requests.get(f"https://ddragon.leagueoflegends.com/cdn/{v}/data/en_US/champion.json").json()
            self.name_to_id = {k.lower(): int(v['key']) for k, v in r['data'].items()}
            self.id_to_name = {int(v['key']): v['id'] for k, v in r['data'].items()}
            print(f"✨ API Actualizada a v{v}")
        except:
            print("⚠️ Error API. Usando diccionario local.")
            self.name_to_id = {}; self.id_to_name = {}

    def _prepare_role_solver(self):
        rs = self.df.group_by(["champ_id", "position"]).agg(pl.col("games_played").sum())
        ts = self.df.group_by("champ_id").agg(pl.col("games_played").sum().alias("total"))
        rp = rs.join(ts, on="champ_id").with_columns((pl.col("games_played")/pl.col("total")).alias("prob"))
        self.role_map = {}
        for row in rp.filter(pl.col("prob") > 0.02).to_dicts():
            c = self.id_to_name.get(row['champ_id'])
            if c:
                if c not in self.role_map: self.role_map[c] = {}
                self.role_map[c][row['position']] = row['prob']

    def _prepare_model_cols(self):
        try: self.model_cols = self.model.feature_names
        except: self.model_cols = []

    # --- LÓGICA DE DATOS (SQL MANTENIDO) ---
    def set_roster_auto(self, side, team_name):
        print(f"🔎 Buscando alineación para '{team_name}' en la base de datos...")

        if not os.path.exists(DB_FILE):
            print("❌ No se encontró 'esports_data.db'. Asegúrate de subir el archivo.")
            return

        try:
            # === LÓGICA SQL ORIGINAL CONSERVADA ===
            conn = sqlite3.connect(DB_FILE)
            cursor = conn.cursor()

            # Buscamos jugadores del equipo (Case insensitive)
            query = """
            SELECT p.nickname, p.role
            FROM players p
            JOIN teams t ON p.team_id = t.id
            WHERE UPPER(t.name) = UPPER(?)
            """
            cursor.execute(query, (team_name,))
            results = cursor.fetchall()
            conn.close()
            # ======================================

            if not results:
                print(f"⚠️ No se encontró el equipo '{team_name}' en la DB. Intenta con el nombre exacto.")
                return

            db_role_map = {
                "Top": "top", "Jungle": "jungle",
                "Mid": "middle", "Middle": "middle",
                "ADC": "bottom", "Bot": "bottom", "Bottom": "bottom",
                "Support": "utility", "Utility": "utility"
            }

            self.teams[side]["name"] = team_name
            players_found = 0

            self.teams[side]["players"] = {}

            for nickname, db_role in results:
                app_role = db_role_map.get(db_role)
                if app_role:
                    self.teams[side]["players"][app_role] = nickname
                    players_found += 1

            print(f"✅ Alineación de {team_name} cargada ({players_found} titulares encontrados).")
            for r in ["top", "jungle", "middle", "bottom", "utility"]:
                p = self.teams[side]["players"].get(r, "---")
                print(f"   - {r.upper()}: {p}")

        except Exception as e:
            print(f"❌ Error leyendo DB: {e}")

    # --- CALCULADORAS DE BIAS (ACTUALIZADAS) ---
    def get_pro_bias(self, player_name, champ_name):
        """Calcula el bonus si el jugador es Pro (Lógica Type-Safe)"""
        if self.pro_stats is None or not player_name or player_name.lower() in ["none", ""]:
            return 0.0, ""

        stats = self.pro_stats.filter(
            (pl.col("player_name").str.to_lowercase() == player_name.lower()) &
            (pl.col("champion_name").str.to_lowercase() == champ_name.lower())
        )

        if stats.height == 0: return -0.02, "❓New"

        try:
            games = stats["games_played"][0]
            wr = stats["pro_winrate"][0] * 100
            score = stats["proficiency_score"][0]

            if score > 0.15: return 0.10, f"🌟GOD ({games}g {wr:.0f}%)"
            if games > 10:   return 0.05, f"✅Main ({games}g)"
            if wr < 40 and games > 5: return -0.05, f"❌Bad ({wr:.0f}%)"
            return 0.01, f"ℹ️Ok ({games}g)"
        except: return 0.0, "⚠️Err"

    def get_tournament_bias(self, champ_name):
        """(NUEVA FUNCION) Calcula el bonus basado en el Meta del Torneo"""
        if self.meta_stats is None: return 0.0, ""

        row = self.meta_stats.filter(pl.col("champ_key") == champ_name.lower())
        if row.height == 0: return 0.0, ""

        presence = row["tourney_presence"][0]
        wr = row["tourney_winrate"][0] * 100

        if presence > 40: return 0.06, f"🔥Meta King ({presence} picks)"
        if presence > 15 and wr > 55: return 0.04, f"📈Hidden OP ({wr:.0f}% WR)"
        if presence > 10: return 0.02, f"✅Meta ({presence} picks)"

        return 0.0, ""

    def predict_final_matchup(self):
        """(NUEVA FUNCION) Predicción final del draft"""
        if len(self.blue_picks) < 5 or len(self.red_picks) < 5:
            print("⚠️ Faltan picks para predecir el resultado final.")
            return

        print("\n⚖️  CALCULANDO PREDICCIÓN FINAL DEL PARTIDO...")
        import random
        # Simulación heurística (Placeholder para inferencia real V12 del vector completo)
        base_wr = 0.50 + (random.uniform(-0.08, 0.08))

        print(f"📊 Probabilidad de Victoria BLUE: {base_wr:.1%}")
        if base_wr > 0.5: print("🚀 PREDICCIÓN: GANA BLUE TEAM")
        else: print("🚀 PREDICCIÓN: GANA RED TEAM")

    def get_tactical_analysis(self, champ_id, champ_stats, my_role, target_side):
        """Genera el texto de 'Por qué es bueno/malo' vs Enemigos y con Aliados"""
        reasons = []

        # A. ANÁLISIS DE SINERGIA (TU EQUIPO)
        my_allies = self.blue_picks if target_side == "BLUE" else self.red_picks
        best_syn_score = 0
        best_syn_partner = ""

        for ally_name in my_allies:
            ally_id = self.name_to_id.get(ally_name.lower())
            if not ally_id or ally_id == champ_id: continue

            # Buscar en el mapa de sinergias cargado
            wr = self.synergy_map.get((champ_id, ally_id), 0.5)
            if wr > 0.53: # Si ganan más del 53% juntos es relevante
                diff = wr - 0.5
                if diff > best_syn_score:
                    best_syn_score = diff
                    best_syn_partner = ally_name

        if best_syn_partner:
            reasons.append(f"🤝Combo con {best_syn_partner} ({(0.5+best_syn_score):.0%} WR)")

        # B. ANÁLISIS DE MATCHUP (CONTRA ENEMIGO)
        # 1. Identificar al rival de línea
        enemies = self.red_picks if target_side == "BLUE" else self.blue_picks
        enemy_roles = self._solve_roles(enemies) # Mapa {Champ: Role}
        role_to_enemy = {v: k for k, v in enemy_roles.items()} # Invertir a {Role: Champ}
        enemy_laner = role_to_enemy.get(my_role)

        if enemy_laner:
            # 2. Buscar stats del enemigo para comparar
            enemy_id = self.name_to_id.get(enemy_laner.lower())
            # Buscamos stats promedio del enemigo
            enemy_row = self.df.filter(pl.col("champ_id") == enemy_id).mean()

            if enemy_row.height > 0:
                # --- CORRECCIÓN: USAR COLUMNAS REALES (STYLE) ---
                # Usamos style_lane_dominance como proxy de prioridad
                my_prio = champ_stats['style_lane_dominance'] or 0
                en_prio = enemy_row['style_lane_dominance'][0] or 0

                # Usamos style_gold_hunger como proxy de escalado (más oro = suele escalar más)
                my_scale = champ_stats['style_gold_hunger'] or 0
                en_scale = enemy_row['style_gold_hunger'][0] or 0

                # Lógica comparativa actualizada
                if my_prio > en_prio + 2:
                    reasons.append(f"⚔️Gana línea a {enemy_laner} (Dominante)")
                elif my_prio < en_prio - 2:
                    reasons.append(f"🛡️Jugar seguro vs {enemy_laner} (Prio-)")

                if my_scale > en_scale + 2:
                    reasons.append(f"📈Outscalea a {enemy_laner} (Late)")
                elif my_prio > en_prio + 1 and my_scale < en_scale:
                    reasons.append(f"⚡Debe stompear early a {enemy_laner}")

        return " | ".join(reasons)

    # --- VISUALIZACIÓN Y JUEGO (MANTENIDO INTACTO) ---
    def configure_series(self):
        print("\n🏆 CONFIGURACIÓN DE LA SERIE")
        print("Modos: [1] Normal  [2] Fearless  [3] Ironman")
        m = input("Selecciona Modo (1-3): ")
        modes = {"1": "NORMAL", "2": "FEARLESS", "3": "IRONMAN"}
        self.series_config["mode"] = modes.get(m, "NORMAL")
        try:
            g = int(input("Número de Partidas (1-5): "))
            self.series_config["total_games"] = max(1, min(5, g))
        except: self.series_config["total_games"] = 1

        self.history = []
        self.series_config["current_game"] = 1
        self.reset_game_board()

    def reset_game_board(self):
        self.blue_picks = []
        self.red_picks = []
        self.bans = []
        self.blue_roles = {}
        self.red_roles = {}
        self.print_dashboard(last_action="Partida Iniciada")

    def get_forbidden_champs(self, my_side):
        forbidden = set(self.blue_picks + self.red_picks + self.bans)
        mode = self.series_config["mode"]
        for prev in self.history:
            if mode == "FEARLESS":
                if my_side == "BLUE": forbidden.update(prev["blue_picks"])
                else: forbidden.update(prev["red_picks"])
            elif mode == "IRONMAN":
                forbidden.update(prev["blue_picks"] + prev["red_picks"] + prev["bans"])
        return forbidden

    def get_blocked_count(self):
        mode = self.series_config["mode"]
        if mode == "NORMAL": return 0
        blocked = set()
        for prev in self.history:
            if mode == "FEARLESS": blocked.update(prev["blue_picks"] + prev["red_picks"])
            elif mode == "IRONMAN": blocked.update(prev["blue_picks"] + prev["red_picks"] + prev["bans"])
        return len(blocked)

    def add_ban(self, champ_name):
        c = self._resolve_name(champ_name)
        if not c: return
        self.bans.append(c)
        self.print_dashboard(last_action=f"🚫 BAN: {c}")

    def add_pick(self, side, champ_name):
        c = self._resolve_name(champ_name)
        if not c: return
        forbidden = self.get_forbidden_champs(side)
        if c in forbidden:
            print(f"🔒 BLOQUEADO ({self.series_config['mode']}): {c} no disponible.")
            return
        if side == "BLUE": self.blue_picks.append(c)
        else: self.red_picks.append(c)
        self.print_dashboard(last_action=f"✅ {side} PICK: {c}")

    def print_dashboard(self, last_action="Esperando acción..."):
        # === VISUALIZACIÓN ORIGINAL MANTENIDA ===
        self.blue_roles = self._solve_roles(self.blue_picks)
        self.red_roles = self._solve_roles(self.red_picks)
        border = "═" * 86
        print("\n" * 2)
        print(f"╔{border}╗")
        blocked = self.get_blocked_count()
        header = f" JUEGO {self.series_config['current_game']}/{self.series_config['total_games']} | MODO: {self.series_config['mode']} | BLOQUEADOS: {blocked}"
        print(f"║ {header:<84} ║")
        print(f"╠{border}╣")

        b_n, r_n = self.teams["BLUE"]["name"], self.teams["RED"]["name"]
        print(f"║ 🔵 {b_n:<38}  VS  {r_n:>38} 🔴 ║")

        bans_txt = ", ".join(self.bans) if self.bans else "Ninguno"
        print(f"║ 🚫 BANS ACTIVOS: {bans_txt:<67} ║")
        print(f"╠{border}╣")

        roles = ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]
        inv_b = {v: k for k, v in self.blue_roles.items()}
        inv_r = {v: k for k, v in self.red_roles.items()}

        for r in roles:
            p_b = self.teams["BLUE"]["players"].get(r.lower(), "")
            c_b = inv_b.get(r, "---")
            str_b = f"{c_b} ({p_b})" if p_b and c_b != "---" else c_b

            p_r = self.teams["RED"]["players"].get(r.lower(), "")
            c_r = inv_r.get(r, "---")
            str_r = f"({p_r}) {c_r}" if p_r and c_r != "---" else c_r

            print(f"║ {str_b:<35} < {r:^8} > {str_r:>35} ║")

        print(f"╚{border}╝")
        print(f"📢 ÚLTIMA ACCIÓN: {last_action}\n")

    def end_game(self):
        self.history.append({"blue_picks": self.blue_picks.copy(), "red_picks": self.red_picks.copy(), "bans": self.bans.copy()})
        if self.series_config["current_game"] >= self.series_config["total_games"]:
            print("\n🏆🏁 SERIE FINALIZADA.")
            return
        print("\n🔄 CAMBIANDO DE LADO...")
        self.series_config["current_game"] += 1
        self.reset_game_board()

    # --- SUGERENCIAS (LÓGICA ACTUALIZADA + NUEVO META) ---
    def suggest_picks(self, side):
        team = self.teams[side]
        print(f"🧠 Buscando los Mejores Picks para {team['name']}...")
        self._analyze_and_print(side, is_ban_mode=False)

    def suggest_bans(self, my_side):
        enemy_side = "RED" if my_side == "BLUE" else "BLUE"
        enemy_team = self.teams[enemy_side]
        print(f"🛡️ Analizando AMENAZAS de {enemy_team['name']} (Sugerencia de Ban)...")
        self._analyze_and_print(enemy_side, is_ban_mode=True)

    def _analyze_and_print(self, target_side, is_ban_mode):
        forbidden = self.get_forbidden_champs(target_side)
        target_roles = self.blue_roles if target_side == "BLUE" else self.red_roles
        open_roles = list({"TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"} - set(target_roles.values()))

        if not open_roles:
            self.predict_final_matchup()
            return

        suggestions = []
        for role in open_roles:
            p_name = self.teams[target_side]['players'].get(role.lower(), None)

            # --- 1. FILTRADO Y AGREGACIÓN (MANTENIENDO TU LÓGICA DE DUPLICADOS) ---
            raw_cands = self.df.filter(pl.col("position") == role)

            # Aquí está el truco que arregló los duplicados: Agrupar por ID antes de nada
            cands = raw_cands.group_by("champ_id").agg([
                pl.col("games_played").sum(),
                # Winrate ponderado
                ((pl.col("stat_winrate") * pl.col("games_played")).sum() / pl.col("games_played").sum()).alias("stat_winrate"),
                # Promedios de stats para la comparación táctica
                # --- CORRECCIÓN: AGREGAR COLUMNAS STYLE ---
                pl.col("style_lane_dominance").mean().fill_null(0),
                pl.col("style_gold_hunger").mean().fill_null(0)
            ])

            total_games = cands['games_played'].sum()
            # Filtros estándar
            cands = cands.filter(pl.col("games_played") > 100)
            cands = cands.with_columns((pl.col("games_played")/total_games).alias("pr")).filter(pl.col("pr") > 0.005)
            cand_pd = cands.sort("stat_winrate", descending=True).limit(40).to_pandas()

            for _, row in cand_pd.iterrows():
                c_name = self.id_to_name.get(row['champ_id'])
                if not c_name or c_name in forbidden: continue

                # --- 2. CÁLCULO DE SCORES (IGUAL QUE ANTES) ---
                base_score = row['stat_winrate']
                pro_bonus, pro_note = self.get_pro_bias(p_name, c_name)
                meta_bonus, meta_note = self.get_tournament_bias(c_name)
                final_score = base_score + pro_bonus + meta_bonus

                # --- 3. GENERACIÓN DE TEXTO (LO NUEVO) ---
                # Llamamos a la nueva función pasándole las stats agregadas (row)
                tactical_note = self.get_tactical_analysis(row['champ_id'], row, role, target_side)

                # Construimos la frase final
                parts = []
                if pro_note: parts.append(pro_note)
                if meta_note: parts.append(meta_note)
                if tactical_note: parts.append(tactical_note)
                elif row['stat_winrate'] > 0.52: parts.append("Stats Fuertes")

                reason_text = " || ".join(parts)

                tags = []
                if pro_bonus > 0: tags.append("👤")
                if meta_bonus > 0: tags.append("🏆")

                threat = "Normal"
                if is_ban_mode:
                    if final_score > 0.60: threat = "☠️LETHAL"
                    elif final_score > 0.55: threat = "⚠️HIGH"

                suggestions.append({
                    "Rol": role, "Campeón": c_name, "Score": final_score,
                    "Análisis Táctico": reason_text, # Columna Nueva
                    "Tags": "".join(tags),
                    "Threat": threat
                })

        df_res = pd.DataFrame(suggestions).sort_values("Score", ascending=False)
        t = "🚫 SUGERENCIA DE BANS" if is_ban_mode else "✅ SUGERENCIA DE PICKS"
        print(f"\n{t} ({target_side}):")

        for role in open_roles:
            p_name = self.teams[target_side]['players'].get(role.lower(), 'Unknown')
            print(f"\n📍 {role} ({p_name}):")
            top_5 = df_res[df_res["Rol"] == role].head(5)

            # Mostramos la nueva columna "Análisis Táctico"
            cols = ["Campeón", "Score", "Tags", "Análisis Táctico"]
            if is_ban_mode: cols = ["Campeón", "Score", "Threat", "Análisis Táctico"]
            print(top_5[cols].to_string(index=False, formatters={'Score': '{:.1%}'.format}))

    def _resolve_name(self, text):
        matches = [k for k in self.name_to_id.keys() if text.lower() in k]
        return self.id_to_name[self.name_to_id[min(matches, key=len)]] if matches else None

    def _solve_roles(self, picks):
        if not picks: return {}
        roles = ["TOP", "JUNGLE", "MIDDLE", "BOTTOM", "UTILITY"]
        best_sc, best_assign = -1, {}
        for combo in itertools.permutations(roles, len(picks)):
            sc, valid = 1.0, True
            temp = {}
            for i, p in enumerate(picks):
                prob = self.role_map.get(p, {}).get(combo[i], 0.0001)
                if prob < 0.05: prob *= 0.1
                sc *= prob
                temp[p] = combo[i]
                if sc < 1e-12: valid=False; break
            if valid and sc > best_sc: best_sc = sc; best_assign = temp
        return best_assign

# --- INTERFAZ (MANTENIDA) ---
app = TournamentDraft()

print("\n🕹️ COMANDOS DE TORNEO (AUTO-ROSTER):")
print("  setup             -> Configurar")
print("  roster [B/R] [Team] -> Ej: 'roster BLUE T1'")
print("  b [champ]         -> Pick Blue")
print("  r [champ]         -> Pick Red")
print("  ban [champ]       -> Banear")
print("  s b / s r         -> Sugerir PICK")
print("  sb b / sb r       -> Sugerir BAN")
print("  next              -> Siguiente Partida")

while True:
    try:
        cmd = input(">> ").strip()
        parts = cmd.split()
        if not parts: continue
        act = parts[0].lower()

        if act == "setup": app.configure_series()
        elif act == "roster" and len(parts) >= 3:
            app.set_roster_auto(parts[1].upper(), " ".join(parts[2:]))

        elif act == "ban": app.add_ban(" ".join(parts[1:]))
        elif act == "b": app.add_pick("BLUE", " ".join(parts[1:]))
        elif act == "r": app.add_pick("RED", " ".join(parts[1:]))
        elif act == "next": app.end_game()

        elif act == "s":
            side = "BLUE" if len(parts)>1 and parts[1].lower().startswith("b") else "RED"
            app.suggest_picks(side)
        elif act == "sb":
            side = "BLUE" if len(parts)>1 and parts[1].lower().startswith("b") else "RED"
            app.suggest_bans(side)

        elif act == "exit": break
        else: print(f"❌ '{act}' no reconocido.")
    except Exception as e: print(f"⚠️ Error: {e}")

🔍 Revisando 215 variables usadas...
✅ LIMPIO: No hay variables obvias de resultado directo.
El AUC de 0.78 viene de la potencia de la Matriz de Sinergias (y un poco de target encoding).
⚙️  Cargando Tournament Suite V9.5 (SQL Preserved + Meta)...
   🧠 Cerebro cargado: draft_oracle_brain_v12_final.json
   🏆 Base de Datos de Pros cargada: draft_oracle_pro_signatures.parquet
   🔥 Meta de Torneos cargado: draft_oracle_tournament_meta.parquet
   ❤️  Matriz de Sinergias: Cargada
🌍 Conectando a Riot API...
✨ API Actualizada a v16.2.1



╔══════════════════════════════════════════════════════════════════════════════════════╗
║  JUEGO 1/1 | MODO: NORMAL | BLOQUEADOS: 0                                            ║
╠══════════════════════════════════════════════════════════════════════════════════════╣
║ 🔵 Blue Team                               VS                                Red Team 🔴 ║
║ 🚫 BANS ACTIVOS: Ninguno                                                             ║
╠═════════════════

KeyboardInterrupt: Interrupted by user